### Objective

In this notebook, we attempt to build surrogate models for predicting the thermal resistance. 

Here, we aim to train GP adaptively to approximate the Tjmax=175 limit state.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc, norm
import os
import sys

from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, max_error, brier_score_loss
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors

import gpflow
import tensorflow as tf
import tensorflow_probability as tfp

### 1. Load dataset

In [2]:
df = pd.read_csv('./Dataset/TcTj_train.csv', header=None)
df.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Pool: {df.shape[0]}")

Pool: 9421


In [3]:
# Remove outliers
df = df[df.Tj<250].reset_index(drop=True)
print(f"Filtered pol: {df.shape[0]}")

Filtered pol: 9386


In [4]:
# Dedicated testing set
df_test = pd.read_csv('./Dataset/TcTj_test.csv', header=None)
df_test.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']

# Remove outliers
df_test = df_test[df_test.Tj<250].reset_index(drop=True)
print(f"Filtered testing pol: {df_test.shape[0]}")

Filtered testing pol: 9375


In [5]:
def create_samples(df, train_num):
   
    # Create dataset
    X = df.iloc[:, :-3].to_numpy()
    y = df.iloc[:, -2].to_numpy()
    
    # Train-test split
    if train_num < len(df):
        test_size = 1-train_num/len(df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    else:
        X_train, y_train = X, y
        X_test, y_test = None, None
    
    return X_train, X_test, y_train, y_test

In [6]:
# Train-test split
X, _, y, _ = create_samples(df, 9000)
X_test, _, y_test, _ = create_samples(df_test, 9000)

# Normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

### 2. Training

In [7]:
def evaluate_model(y_true, y_pred):
    """This function is used for evaluating the ML models performance."""
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    max_e = max_error(y_true, y_pred)
    
    percentage = np.abs(y_true-y_pred)/y_true
    max_percentage = np.max(percentage)*100
    max_percentage_loc = np.argmax(percentage)
    mean_percentage = np.mean(percentage)*100
    
    return rmse, max_e, max_percentage, max_percentage_loc, mean_percentage

#### GPflow setup

In [8]:
def init_length_scales(dim, n_restarts, initial_guess=None):
    
    # Random initial params
    lb, ub = -2, 2
    lhd = qmc.LatinHypercube(d=dim, seed=42).random(n_restarts)
    lhd = (ub-lb)*lhd + lb
    length_scales = 10**lhd

    # Informed initial guess
    if initial_guess is not None:
        length_scales = np.vstack((length_scales, initial_guess))

    return length_scales

In [9]:
def fit(X, y, n_restarts=20, init_lengthscales=None, init_variance=None, trainable=True, verbose=True):
    models = []
    log_likelihoods = []
    
    # Generate initial guesses for length scale
    length_scales = init_length_scales(X.shape[1], n_restarts, init_lengthscales)
    if init_variance is None:
        variance=np.var(y_train)
    else:
        variance=init_variance

    if not trainable:
        model = gpflow.models.GPR(
            (X, y.reshape(-1, 1)),
            kernel=gpflow.kernels.SquaredExponential(variance=variance, lengthscales=init_lengthscales),
            mean_function=gpflow.functions.Polynomial(0),
        )

        return model

    else:
        with tf.device("CPU:0"):
            
            for i, init in enumerate(length_scales):
                
                if verbose:
                    print(f"Performing {i+1}-th optimization:")
    
                # Set up the model
                kernel = gpflow.kernels.SquaredExponential(variance=variance, lengthscales=init)
                model = gpflow.models.GPR(
                    (X, y.reshape(-1, 1)),
                    kernel=kernel,
                    mean_function=gpflow.functions.Polynomial(0),
                )
    
                opt = gpflow.optimizers.Scipy()
                opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=100))
    
                models.append(model)
                log_likelihoods.append(model.log_marginal_likelihood().numpy())
    
        # Select the model with the highest log-marginal likelihood
        best_model_index = np.argmax(log_likelihoods)
        best_model = models[best_model_index]

        return best_model

#### Active learning

Select diverse batch

In [10]:
def select_diverse_batch(samples, acq, batch_size=5, pre_filter=False):
    
    if pre_filter:
        thred = np.quantile(acq, pre_filter)
        filtered_indices = np.arange(len(samples))[acq>thred]
        samples = samples[acq>thred]
        acq = acq[acq>thred]
    
    else:
        filtered_indices = np.arange(len(samples))
        
    # Perform weighted K-means clustering on the samples
    kmeans = KMeans(n_clusters=batch_size, n_init=10, random_state=0).fit(samples, sample_weight=acq)
    cluster_labels = kmeans.labels_

    # Find the highest acquisition value sample in each cluster
    selected_indices = []
    for cluster_idx in range(batch_size):
        cluster_indices = np.where(cluster_labels == cluster_idx)[0]
        cluster_acquisition_values = acq[cluster_indices]
        best_index_in_cluster = cluster_indices[np.argmax(cluster_acquisition_values)]
        selected_indices.append(best_index_in_cluster)

    return filtered_indices[selected_indices]

Acquisition function

In [11]:
def acquisition(model, candidate, limit_state_value=0, batch_mode=False, batch_size=None):

    # Compute prediction variance
    f_mean, f_var = model.predict_f(candidate, full_cov=False)
    f_mean = f_mean.numpy().flatten()
    f_var = f_var.numpy().flatten()

    # Calculate U values
    U_values = np.abs(f_mean-limit_state_value)/np.sqrt(f_var)

    # Sample selection
    if batch_mode:
        # Batch selection mode
        U_normalied = MinMaxScaler().fit_transform(1/U_values.reshape(-1, 1))
        indices = select_diverse_batch(candidate, U_normalied.flatten(), batch_size=batch_size)
    
    else:
        # Single point selection mode
        indices = np.argmin(U_values)

    return U_values, indices

In [12]:
def sample_creator(X, y, sample_num, sampling_scheme='LHS'):
    
    # Create virtual samples
    if sampling_scheme == 'LHS':
        raw_virtual_samples = qmc.LatinHypercube(d=X.shape[1]).random(n=sample_num)
    elif sampling_scheme == 'Halton':
        raw_virtual_samples = qmc.Halton(d=X.shape[1]).random(sample_num)
    else:
        raise ValueError(f"Invalid sampling scheme: {sampling_scheme}")

    # Dataset statistics
    X_scaled = MinMaxScaler().fit_transform(X)

    # Find closest real samples
    sample_finder = NearestNeighbors(n_neighbors=1).fit(X_scaled)
    _, indices = sample_finder.kneighbors(raw_virtual_samples)
    
    # Drop duplicates
    indices = np.unique(indices)

    # Select samples
    selected_X = X[indices.flatten()]
    selected_y = y[indices.flatten()]

    # Identify remaining samples
    all_indices = np.arange(X.shape[0])
    remaining_indices = np.setdiff1d(all_indices, indices)
    remaining_X = X[remaining_indices]
    remaining_y = y[remaining_indices]
  
    return selected_X, selected_y, remaining_X, remaining_y

In [13]:
def confidence_assessment(model, X_test, y_test, limit_state):
    f_mean, f_var = model.predict_f(X_test, full_cov=False)
    y_prob = norm.cdf(limit_state, loc=f_mean, scale=np.sqrt(f_var))
    label = np.where(y_test > limit_state, 1, 0)
    brier_score = brier_score_loss(label, 1-y_prob)
    return brier_score

In [14]:
# Initial samples 
ini_sample_num = 300
X_train, y_train, X_pool, y_pool = sample_creator(X, y, ini_sample_num, sampling_scheme='Halton')
X_train_scaled = scaler.transform(X_train)

In [15]:
n_iter = 100
U_hist, test_brier_scores = [], []
Tjmax = 175

for i in range(n_iter):
    print(f"Start {i+1}th learning iteration:")

    # 1-GP model training and predicting
    if i == 0:
        model = fit(X_train_scaled, y_train, n_restarts=20, verbose=False)

    else:
        # model = fit(X_train_scaled, y_train, n_restarts=20, verbose=False)
        model = fit(X_train_scaled, y_train, n_restarts=5, init_lengthscales=init_lengthscales, 
                    init_variance=init_variance, verbose=False)

    # 2-Check fitting results
    f_mean, _ = model.predict_f(scaler.transform(X_test), full_cov=False)
    has_nan = tf.reduce_any(tf.math.is_nan(f_mean)).numpy()
    counter = 0
    while has_nan:
        print(f"Bad fitting. Refit the data:")
        counter += 1
        model = fit(X_train_scaled, y_train, n_restarts=5, verbose=False)
        f_mean, _ = model.predict_f(scaler.transform(X_test), full_cov=False)
        has_nan = tf.reduce_any(tf.math.is_nan(f_mean)).numpy()

        if counter > 4:
            print(f"Fallback to parameters from last iteration:")
            model = fit(X, y, n_restarts=1, init_lengthscales=init_lengthscales.flatten(), 
                        init_variance=init_variance, trainable=False, verbose=False)
            f_mean, _ = model.predict_f(scaler.transform(X_test), full_cov=False)
            has_nan = tf.reduce_any(tf.math.is_nan(f_mean)).numpy()

    print(f"Good fitting. Proceed:")

    # 3-Model assessment
    brier_score = confidence_assessment(model, scaler.transform(X_test), y_test, Tjmax)
    test_brier_scores.append(brier_score)

    # 4-Acquisition
    X_pool_scaled = scaler.transform(X_pool)
    U_values, indices = acquisition(model, X_pool_scaled, limit_state_value=175, batch_mode=True, batch_size=10)
    target = np.min(U_values[indices])
    U_hist.append(target)
    print(f"Iter {i+1}: test brier score==>{brier_score:.5f}/{0.01039}, U==>{target:.4f}/1.65, index==>{indices}")

    if target >= 2:
        break

    # 5-Updating
    X_train = np.vstack((X_train, X_pool[indices]))
    X_train_scaled = scaler.transform(X_train)
    y_train = np.append(y_train, y_pool[indices])

    # Update pool
    X_pool = np.delete(X_pool, obj=indices, axis=0)
    y_pool = np.delete(y_pool, obj=indices, axis=0)

    # Update initial guess
    init_lengthscales = model.kernel.lengthscales.numpy().reshape(1, -1)
    init_variance = model.kernel.variance.numpy().flatten()[0]

Start 1th learning iteration:
Good fitting. Proceed:
Iter 1: test brier score==>0.01497/0.01039, U==>0.0002/1.65, index==>[ 655 3142  801   66 4568 1605 6252  880 7363 5351]
Start 2th learning iteration:
Good fitting. Proceed:
Iter 2: test brier score==>0.01427/0.01039, U==>0.0142/1.65, index==>[4185 2733 6231 4083 3577 6412 2490 1304 6560 2103]
Start 3th learning iteration:
Good fitting. Proceed:
Iter 3: test brier score==>0.01544/0.01039, U==>0.0115/1.65, index==>[1662 3726 1472 1948 1151 4225 6495 5165 4585 3840]
Start 4th learning iteration:
Bad fitting. Refit the data:
Good fitting. Proceed:
Iter 4: test brier score==>0.01487/0.01039, U==>0.0101/1.65, index==>[1495  437 1235 3561 8579 2957 4755 5338 4212 6924]
Start 5th learning iteration:
Good fitting. Proceed:
Iter 5: test brier score==>0.01482/0.01039, U==>0.0049/1.65, index==>[3833  998 7383 5034 7205 1377 5515 4988 6017 5614]
Start 6th learning iteration:
Good fitting. Proceed:
Iter 6: test brier score==>0.01421/0.01039, U==>

In [23]:
# Save model
import pickle
with open('AL_model_params.pickle', 'wb') as handle:
    pickle.dump(gpflow.utilities.parameter_dict(model), handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save training data
np.save('AL_X_train.npy', X_train)
np.save('AL_y_train.npy', y_train)

# Save history
df = pd.DataFrame({"U": np.array(U_hist), "brier_scores": np.array(test_brier_scores)})
df['benchmark'] = 0.01039
df.to_csv("AL_history.csv", index=False)

#### Propose solutions

In [24]:
df_candidates = pd.read_csv('./Dataset/candidates.csv')
df_candidates.columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
print(f"PCandidate pool: {df_candidates.shape[0]}")

PCandidate pool: 470010


In [25]:
def evaluate_weight(X):
    # Properties
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(X[:, 3] / Fan_height)

    # Weight calculation
    w = density_Al*(X[:, 3]*X[:, 2]*X[:, 4]+X[:, 7]*(X[:, 5]*X[:, 8]*X[:, 4]))+ Fan_Weight*N_fan

    return w

In [26]:
Q_df = pd.read_csv('./Dataset/Q_test_locations.csv')

In [27]:
for i, (Q1, Q2) in enumerate(zip(Q_df['Q1'].to_numpy(), Q_df['Q2'].to_numpy())):

    print(f"Handling {i+1}th condition:")
    
    # Compile feature samples
    Q1_array, Q2_array = Q1*np.ones((df_candidates.shape[0], 1)), Q2*np.ones((df_candidates.shape[0], 1))
    X_candidates = df_candidates.to_numpy()
    X_candidates = np.hstack((Q1_array, Q2_array, X_candidates))
    X_candidates_scaled = scaler.transform(X_candidates)

    # GP prediction
    f_mean, f_var = model.predict_f(X_candidates_scaled, full_cov=False)
    f_mean = f_mean.numpy().flatten()
    f_var = f_var.numpy().flatten()

    # Utility
    Tjmax = 175
    likelihood = norm.cdf(Tjmax, loc=f_mean, scale=np.sqrt(f_var))
    w = evaluate_weight(X_candidates)
    utility = likelihood*1/w

    # Sort candidates
    df = pd.DataFrame(X_candidates)
    df.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
    df['weight'] = w
    df['pred_T'] = f_mean
    df['utility'] = utility
    df_sorted = df.sort_values(by='utility', ascending=False).reset_index(drop=True)

    # Output results
    df_reduced = df_sorted.iloc[:20, :].reset_index(drop=True)
    df_reduced.to_csv(f"Exp_{i+1}.csv", index=False)

Handling 1th condition:
Handling 2th condition:
Handling 3th condition:
Handling 4th condition:
Handling 5th condition:
Handling 6th condition:
Handling 7th condition:
Handling 8th condition:
Handling 9th condition:
Handling 10th condition:
Handling 11th condition:
Handling 12th condition:
Handling 13th condition:
Handling 14th condition:
Handling 15th condition:
Handling 16th condition:
Handling 17th condition:
Handling 18th condition:
Handling 19th condition:
Handling 20th condition:
Handling 21th condition:
Handling 22th condition:
Handling 23th condition:
Handling 24th condition:
Handling 25th condition:
Handling 26th condition:
Handling 27th condition:
Handling 28th condition:
Handling 29th condition:
Handling 30th condition:
Handling 31th condition:
Handling 32th condition:
Handling 33th condition:
Handling 34th condition:
Handling 35th condition:
Handling 36th condition:
Handling 37th condition:
Handling 38th condition:
Handling 39th condition:
Handling 40th condition:
Handling 